In [1]:
import os
import sys
import git
import pathlib

import random

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

PROJ_ROOT_PATH = pathlib.Path(git.Repo('.', search_parent_directories=True).working_tree_dir)
PROJ_ROOT =  str(PROJ_ROOT_PATH)
if PROJ_ROOT not in sys.path:
    sys.path.append(PROJ_ROOT)

from libs import utils, fashion_cnn2
from libs.constants import model_seeds

from timeit import default_timer as timer

In [2]:
# Limit GPU growth
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [3]:
# load dataset  
fashion_mnist = tf.keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

train_images = train_images.astype('float32') / 255.0
test_images = test_images.astype('float32') / 255.0

train_images = np.expand_dims(train_images, axis=-1)
test_images = np.expand_dims(test_images, axis=-1)

In [4]:
model_type = "fashion-cnn2_1024"
model_seed = model_seeds[0]

In [5]:
# Get model
model_instance = model_type + "-" + str(model_seed)
model_folder = pathlib.Path(PROJ_ROOT_PATH / "models" / model_type)
model_filename = model_instance + ".h5"
model_file = str(pathlib.Path(model_folder/ model_filename))
# Load model
model = tf.keras.models.load_model(model_file)

In [6]:
no_im = 32 # no of images per batch
b_images = test_images[0:no_im]
b_labels = test_labels[0:no_im]
# image = tf.expand_dims(all_images[0],axis=0)

In [7]:
# Load model
model = tf.keras.models.load_model(model_file)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 c0 (Conv2D)                 (None, 28, 28, 32)        544       
                                                                 
 p0 (MaxPooling2D)           (None, 14, 14, 32)        0         
                                                                 
 dropout (Dropout)           (None, 14, 14, 32)        0         
                                                                 
 c1 (Conv2D)                 (None, 14, 14, 32)        16416     
                                                                 
 p1 (MaxPooling2D)           (None, 7, 7, 32)          0         
                                                                 
 dropout_1 (Dropout)         (None, 7, 7, 32)          0         
                                                                 
 flatten (Flatten)           (None, 1568)              0

In [8]:
# testing batch_fashion_cnn2_ERR
from libs.fitnessfns import batch_fashion_cnn2_ERR
error_profile_c0 = None
error_profile_c1 = None
error_profile_h0 = None
error_profile_op = None
ERR_PARAM_TF = None
shuffle_order_c0 = None
shuffle_order_c1 = None
shuffle_order_h0 = None
shuffle_order_op = None
                                        
manual_output = batch_fashion_cnn2_ERR( b_images,
                                        b_labels,
                                        model,
                                        error_profile_c0,
                                        error_profile_c1,
                                        error_profile_h0,
                                        error_profile_op,
                                        ERR_PARAM_TF,
                                        shuffle_order_c0,
                                        shuffle_order_c1,
                                        shuffle_order_h0,
                                        shuffle_order_op)

print(manual_output)

tf.Tensor(2, shape=(), dtype=int64)


In [9]:
# testing ff_fashion_cnn2_ERR
from libs.fitnessfns import ff_fashion_cnn2_ERR
error_profile_c0 = None
error_profile_c1 = None
error_profile_h0 = None
error_profile_op = None
ERR_PARAM = None
shuffle_order_c0 = None
shuffle_order_c1 = None
shuffle_order_h0 = None
shuffle_order_op = None
test_set = (test_images, test_labels)
batchsize = 128
                                        
acc = ff_fashion_cnn2_ERR(model,
                    error_profile_c0,
                    error_profile_c1,
                    error_profile_h0,
                    error_profile_op,
                    ERR_PARAM,
                    shuffle_order_c0,
                    shuffle_order_c1,
                    shuffle_order_h0,
                    shuffle_order_op,
                    test_set,
                    batchsize)

print(acc)

tf.Tensor(0.9258814102564102, shape=(), dtype=float64)


In [10]:
# testing eval_fashion_cnn2_ERR
from libs.fitnessfns import eval_fashion_cnn2_ERR

error_profile_c0 = None
error_profile_c1 = None
error_profile_h0 = None
error_profile_op = None
ERR_PARAM = None
shuffle_order_c0 = None
shuffle_order_c1 = None
shuffle_order_h0 = None
shuffle_order_op = None
test_set = (test_images, test_labels)

acc_mean, acc_std = eval_fashion_cnn2_ERR(model,
                        error_profile_c0,
                        error_profile_c1,
                        error_profile_h0,
                        error_profile_op,
                        ERR_PARAM,
                        shuffle_order_c0,
                        shuffle_order_c1,
                        shuffle_order_h0,
                        shuffle_order_op,
                        test_set)

print(acc_mean, acc_std)

0.9258814102564102 0.0


In [11]:
# testing eval_fashion_cnn2_ERR with dummy error profile and shuffle order
dummy_error_profile = np.zeros((20_000,32), dtype="float32")
error_profile_c0 = dummy_error_profile#None
error_profile_c1 = dummy_error_profile#None
error_profile_h0 = dummy_error_profile#None
error_profile_op = dummy_error_profile#None
shuffle_order_c0 = np.arange(model.get_layer("c0").weights[0].shape[-1])
shuffle_order_c1 = np.arange(model.get_layer("c1").weights[0].shape[-1])
shuffle_order_h0 = np.arange(model.get_layer("h0").weights[0].shape[-1])
shuffle_order_op = np.arange(model.get_layer("op").weights[0].shape[-1])
ERR_PARAM = -1
np.random.shuffle(shuffle_order_c0)
np.random.shuffle(shuffle_order_c1)
np.random.shuffle(shuffle_order_h0)
np.random.shuffle(shuffle_order_op)
test_set = (test_images, test_labels)
acc_mean, acc_std = eval_fashion_cnn2_ERR(model,
                        error_profile_c0,
                        error_profile_c1,
                        error_profile_h0,
                        error_profile_op,
                        ERR_PARAM,
                        shuffle_order_c0,
                        shuffle_order_c1,
                        shuffle_order_h0,
                        shuffle_order_op,
                        test_set)

print(acc_mean, acc_std)

0.9258814102564102 0.0
